In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set(rc={'figure.figsize':(15, 10)})

# Základné operácie

Najprv je potrebné si súbor načítať a preskúmať základné štatistiky pomocou info() a describe()

In [ ]:
labor = pd.read_csv('./data/labor.csv', sep='\t', index_col=0)
labor.head()

In [ ]:
labor.columns

Počet záznamov a premenných môžeme zistiť viacerými spôsobmi:
- pomocou shape, `shape[0]` predstavuje počet záznamov a `shape[1]` predstavuje počet premenných
- pomocou `info()`, ktore vypíše aj ďalšie informácie, ako počet non-null hodnôt, dátové typy premenných a veľkosť datasetu na disku

Vďaka info sme mohli zistiť, ktoré premenné sú numerické (`float64`, `int64`, ...), a ktoré sú kategorické(`object`, `category`, ...):
- numerické: leukocyty, trombocyty, erytrocyty, er-cv, hemoglobin, indicator, hbver, hematokrit, etytr, alt, weight, ast, alp
- name, ssn, relationship, smoker

In [ ]:
print('Pocet zaznamov:', labor.shape[0])
print('Pocet premennych:', labor.shape[1])

In [ ]:
labor.info()

Funkcia `describe()` vypíše základné súhrnné informácie o datasete. Výpisom je tabuľka s počtom, priemerom, odchýlkou, minimom, maximom a kvartilmi pre každú numerickú premennú. Môžeme vidieť, že premenné `er-cv`, `alt`, `ast` a `alp` dosahujú hodnoty medzi 0 a 100, takže zrejme ide o percentuálne hodnoty, `indicator` je binárna premenná a ostatné vypísané sú pravdepodobné normálne numerické premenné.

Ďalej môžeme vidieť, že väčšina premenných má chýbajúce hodnoty - väčšina má ~10 056 non-null záznamov, ale `indicator` má až 10 086 záznamov.

In [ ]:
labor.describe()

# Čistenie dát

Vyššie sme si mohli všimnúť, že premenná `smoker` má viacero hodnôt predstavujúcich to isté, takže je potrebné dáta upraviť.

In [ ]:
labor.smoker.value_counts()

Z kategorickej premennej `smoker` môžeme spraviť numerickú tak, že hodnoty 'yes' a 'Y' budú 1, a hodnoty 'no' a 'N' budú 0.

In [ ]:
labor.smoker.replace({'yes': 1, 'Y': 1, 'no': 0, 'N': 0}, inplace=True)
labor.smoker.value_counts()

In [ ]:
labor.relationship.unique()

Vo vyššom výpise unikátnych hodnôt pre stĺpec s hodnotou `relationship` si môžeme všimnúť, že je v dátach preklep. Namiesto `divoced`, by sme mali mať v dátach ako hodnotu `divorced`. Túto chybu v dátach teda taktiež opravíme 

In [ ]:
labor.relationship.replace({'divoced': 'divorced'}, inplace=True)

V dátach môžu byť ďalšie nezrovnalosti, chýbajúce dáta, duplicitné záznamy a pod.

In [ ]:
print('S duplikatmi:', labor.shape[0])
labor.drop_duplicates(inplace=True)
print('Bez duplikatov:', labor.shape[0])

labor.isna().sum()

Namiesto odstraňovania záznamov s chýbajúcimi hodnotami, môžeme pridať priemer, medián alebo modus danej premennej.

In [ ]:
labor.fillna(labor.mean(), inplace=True)
labor.isna().sum()

In [ ]:
labor.shape

V tabuľke vyprodukovanej pomocou `describe()` môžeme taktiež pozorovať, že váha (`weight`) obsahuje záporné hodnoty, čo samozrejme nie je možné. Dokopy je v datasete 211 záznamov so zápornou váhou.

In [ ]:
labor[labor.weight < 0]

V boxplote však môžeme vidieť, že celá distribúcia je akoby posunutá k záporným hodnotám. Ak predpokladáme, že váha je v librách, priemer váhy sa nachádza niekde pri 70 lb čo je približne 31 kg.

In [ ]:
print('Mean weight:', labor.weight.mean())
labor.weight.plot(kind='box');

Medián veku nakazených je 70 rokov [https://doi.org/10.1016/j.disamonth.2012.01.009], pričom v tejto vekovej kategórii nie je veľa ľudí, ktorí by mali 35 kg. Taktiež maximálna váha v datasete je 210 lb, teda 95 kg, čo je samozrejme nízka váha vzhľadom na vek a geografický pôvod dát. Môžeme teda prehlásiť, že premenná váhy je vychýlená a teda ju musíme opraviť. 

Aby sme vedeli váhu správne upraviť, musíme vedieť o koľko ju treba posunúť. Vďaka datasetu `profiles` vieme, že záznamy pochádzajú zo Spojených štátov, takže si môžeme nájsť priemernú váhu v USA a váhu v datasete posunieme o rozdiel v priemeroch.

Priemerná váha v Severnej Amerike je 80.7 kg, čo je 177.9 lb [https://doi.org/10.1186%2F1471-2458-12-439].

In [ ]:
profiles = pd.read_csv('./data/profiles.csv', sep='\t', index_col=0)
profiles.head()

In [ ]:
diff = 177.9 - labor.weight.mean()
print('Mean difference:', diff)

Rozdiel priemernej váhy populácie a datasetu je takmer 108 lb, čo je približne 49 kg. Teraz zostáva už iba pripočítať rozdiel k datasetu a premenná by mala byť v poriadku.

In [ ]:
labor.weight = labor.weight + diff
labor.weight.plot(kind='box');

# Spájanie datasetov

Druhý dataset je `profiles` s menami, pohlavím, krvnou skupinou, zamestnaním a ďaľsími informáciami o pacientoch. Tabuľky `labor` a `profiles` je možné spojiť na základe mena alebo ssn, najprv si však môžeme všimnúť, že dátumy narodení sú vo viacerých formátoch, takže ich najprv musíme naformátovať.

Premenná `race` obsahuje viacero rovnakých premenných, ktoré sú napísané inak alebo s chybou, takže musíme opraviť aj tie.

In [ ]:
profiles.race.value_counts()

In [ ]:
profiles = profiles.astype({'birthdate': 'datetime64[ns]'})
profiles.race.replace({'white': 'White', 'black': 'Black', 'blsck': 'Black'}, inplace=True)

profiles.head()

In [ ]:
profiles.race.value_counts()

`info()` nám prezradí, že nemáme žiadne chýbajúce informácie o pacientoch a teda môžeme tabuľky spojiť.

In [ ]:
profiles.info()

In [ ]:
data = pd.merge(profiles, labor, on='ssn')
data.head()

Počet záznamov sa nám nezmenil, čo je dobré znamenie, že sme nemali pacientov v `profiles`, ktorí nemajú záznam v `labor`. Taktiež môžeme vidieť, že nám už nechýbajú žiadne záznamy.

In [ ]:
data.shape

In [ ]:
data.isna().sum()

In [ ]:
data.info()

Vidíme, že meno máme v datasete dvakrát, takže jedno môžeme odstrániť

In [ ]:
data.drop('name_y', axis=1, inplace=True)
data.rename({'name_x': 'name'}, axis=1, inplace=True)
data.info()

# Základná analýza premenných

Najprv si pozrieme distribúciu krviniek v krvi. Leukocyty sú biele krvinky, erytrocyty sú červené krvinky a trombocyty sú krvné doštičky. Na grafe môžeme vidieť, že počet bielch a červených krviniek má normálnu distribúciu, ale pri krvných doštičkách môžeme pozorovať distribúciu s dvoma vrcholmi, čo môže napovedať, že dataset je chybný alebo ľudia, ktorí nepotrebujú pravidelné návštevy u doktora majú odlišný počet krvných doštičiek. Túto ideu môžeme ďalej skúmať pri párovej analýze.

In [ ]:
data[['leukocyty', 'erytrocyty', 'trombocyty']].plot(kind='kde');

Aby sme zistili či majú dáta normálne rozdelenie, môžeme použiť aj Shapiro-Wilk test normálnosti. Rozdelenie je normálne ak p-hodnota je vyššia ako zadaná tolerancia, v tomto prípade 0.05, teda 5%. Problémom je, že Shapiro-Wilk test nemusí vrátiť správne výsledky pri viac ako 5000 pozorovaniach, čo môžeme vidieť aj pri erytrocytoch, ktoré majú podľa grafu normálne rozdelenie, ale podľa Shapiro-Wilk testu rozdelenie normálne nie je.

In [ ]:
leukocyty_shapiro = stats.shapiro(data.leukocyty)
erytrocyty_shapiro = stats.shapiro(data.erytrocyty)
trombocyty_shapiro = stats.shapiro(data.trombocyty)

alpha = 0.05
print(leukocyty_shapiro)
if leukocyty_shapiro.pvalue > alpha:
    print('Leukocyty maju normalne rozdelenie\n')
else:
    print('Leukocyty nemaju normalne rozdelenie\n')

print(erytrocyty_shapiro)
if erytrocyty_shapiro.pvalue > alpha:
    print('Erytrocyty maju normalne rozdelenie\n')
else:
    print('Erytrocyty nemaju normalne rozdelenie\n')

print(trombocyty_shapiro)
if trombocyty_shapiro.pvalue > alpha:
    print('Trombocyty maju normalne rozdelenie\n')
else:
    print('Trombocyty nemaju normalne rozdelenie\n')


Môžeme teda použiť iný druh grafu, napríklad Q-Q plot, podľa ktorého vidíme, že leukocyty a erytrocyty majú normálnu distribúciu a trombocyty sú mierne vychýlené.

In [ ]:
import statsmodels.api as sm

fig, ax = plt.subplots(3, 1, figsize=(15, 20))

sm.qqplot(data.leukocyty, ax=ax[0], line='s')
ax[0].set_title('Leukocyty')

sm.qqplot(data.erytrocyty, ax=ax[1], line='s')
ax[1].set_title('Erytrocyty')

sm.qqplot(data.trombocyty, ax=ax[2], line='s')
ax[2].set_title('Trombocyty');

Test normálnosti môžeme spraviť pre všetky numerické premenné v datasete. Vidíme, že normálne rozdelenie majú premenné `leukocyty`, `er-cv`, `hemoglobin`, `weight` a `ast`.

In [ ]:
for col in data.select_dtypes(include='number').columns:
    shapiro_test = stats.shapiro(data[[col]])
    print(col, shapiro_test)

    alpha = 0.05
    if shapiro_test.pvalue > alpha:
        print('>>> Normal distribution (fail to reject H0)\n')
    else:
        print('Another distributions (reject H0)\n')

Vytvoríme si grafy na základné distribúcie u ľudí, ktorí potrebujú pravidelnú návštevu doktora (indicator = 1) a tých, ktorí nepotrebujú (indicator = 0) a v ďalších grafch distribúcie fajčiarov (smoker = 1) a nefajčiarov (smoker = 0). V datasete je viac ľudí, ktorí vyžadujú pravidelné kontroly u doktora a zároveň je v datasete viac nefajčiarov ako fajčiarov.

In [ ]:
data.indicator.value_counts().plot(kind='bar', title="Distribution of patients who need regular examination and those who don't");

In [ ]:
data.smoker.value_counts().plot(kind='bar', title='Distribution of smokers and non-smokers');

Ďalej si môžeme pozrieť distribúciou pacientov podľa rasy. Môžeme vidieť, že najviac pacientov je bielych, pričom druhí v poradí sú čierni s približne polovičným počtom pacientov, potom sú príslušníci Ázijského, Havajského a nakoniec Indiánskeho etnika.

In [ ]:
data.race.value_counts().plot(kind='bar', title='Distribution of ethnicity');

Nakoľko skúmame chorobu prejavujúcu sa zvýšeným počtom leukocytov v krvi, môžeme zisťovať, či niektorá z krvných skupín neprevyšuje počtom prípadov ostatné krvné skupiny. Distribúcia krvných skupín ukazuje, že počty sú približne rovnaké.

In [ ]:
data.blood_group.value_counts().plot(kind='bar', title='Distribution of blood groups');

# Párová analýza dvoch premenných

Je náročné porovnávať náhodné dvojice premenných, ak nevieme, ktoré spolu môžu súvisieť. Preto môžeme vytvoriť pairplot, ktorý nám zobrazí graf každej premennej v korelácii s každou ďalšou premennou v datasete (POZOR: Tento graf je veľký a jeho generovanie môže trvať aj niekoľko minút!)

Binárne premenné `indicator` a `smoker` sa nahromadia na kraje grafov a na diagonále sa zobrazujú distribúcie každej premennej samostatne. Môžeme si všimnúť, že premenná `alt` je pozitívne naklonená a premenná `alp` je negatívne naklonená.

Ďalej môžeme vidieť zhluky, ktoré pre nás nie sú veľmi zaujímavé, až na tie, ktoré vytvárajú nejaký tvar, napríklad krivku, pretože tieto premenné môžu naznačovať koreláciu medzi premennými. Koreláciu môžeme vidieť medzi premennými `alt` a `erytrocyty` a `alp` a `hemoglobin`.

In [ ]:
sns.pairplot(data=data);

Samozrejme, párový graf nie je veľmi prehľadný, takže namiesto neho môžeme použiť korelačnú maticu, kde je oveľa jednoduchšie nájsť korelácie. Taktiež vďaka matici môžeme nájsť negatívnu koreláciu medzi premennou `hbver` a `indicator`, ktorú v párovom grafe nevidíme. Môžeme si však všimnúť, že korelácia medzi premennými `alp` a `hemoglobin` v matici nie je silná, ale v párovom grafe tieto premenné vytvárajú krivku.

In [ ]:
sns.heatmap(data=data.corr(), annot=True, fmt=".3f").set_title('Correlation matrix');

Pozrieme sa teraz bližšie na spomínané dvojice premenných. Spojité premenné najlepšie vizualizujeme pomocou scatterplotu. Vidíme, že s rastúcim množstvom `erytrocytov` v krvi exponenciálne rastie aj premenná `alt`, ale nie hneď, až pri hodnotách 7 a viac. 

Ak si nastavíme pri vizualizácii aj sfarbenie podľa `indikátora`, môžeme vidieť, že tieto premenné pravdepodobne neovplyvňujú či musí pacient chodiť na pravidelné kontroly alebo nie, nakoľko body z oboch skupín sa vyskytujú v celom grafe.

In [ ]:
sns.scatterplot(data=data, x='erytrocyty', y='alt', hue='indicator');

Môžeme si vytvoriť jointplot, ktorý nám ukáže, že `erytrocyty` majú pravdepodobne normálnu distribúciu s priemerom okolo 6 a premenná `alt` je jemne naklonená na jednu stranu a priemerom približne 10%.

In [ ]:
sns.jointplot(data=data, x='erytrocyty', y='alt', kind='reg', order=5, 
              line_kws={'color': 'red'}, height=12);

Ako ďalšiu dvojicu premenných preskúmame vzťah medzi `hemoglobin` a `alp`. Môžeme si všimnúť, že premenné vytvárajú krivku podobnú sinusoide. Hodnoty opäť pravdepodobne neovplyvňujú `indikátor`.

In [ ]:
sns.scatterplot(data=data, x='hemoglobin', y='alp', hue='indicator');

Koreláciu môžeme vidieť aj medzi premennými `hbver` a `hematokrit`, aj keď nie veľmi silnú. Ak si vytvoríme scatterplot s farebným rozlíšením podľa `indikátora`, môžeme vidieť, že nám tam vznikli dva zhluky, ktoré sa však výrazne prekrývajú.

In [ ]:
sns.scatterplot(data=data, x='hematokrit', y='hbver', hue='indicator');

Ďalej môžeme preskúmať vzťah medzi `indikátorom` a premennou `hematokrit`. Indikátor je binárna premenná dosahujúca hodnoty 0 alebo 1, takže s ňou budeme pracovať ako s kategorickou premennou. Tie už nemôžeme vykresliť scatterplotom, takže použijeme boxplot. Okamžite vidíme, že pacienti s nutnou pravidelnou kontrolou majú vyššie hodnoty `hematokritu`.

In [ ]:
sns.boxplot(data=data, x='indicator', y='hematokrit');

Opačný trend môžeme pozorovať pri vizualizácii vzťahu medzi premnnými `indikátor` a `hbver`. V tomto prípade pacienti vyžadujúci pravidelnú kontrolu majú nižšie hodnoty premennej `hbver`.

In [ ]:
sns.boxplot(data=data, x='indicator', y='hbver');

# Hypotézy

## 1. Ovplyvňuje fajčenie telesnú hmotnosť pacienta?

Z datasetu môžeme skúmať aký má vplyv fajčenie na hmotnosť pacientov:

H0: Fajčiari majú v priemere **rovnakú** váhu ako nefajčiari

H1: Fajčiari majú v priemere **vyššiu/nižšiu** váhu ako nefajčiari

Môžeme si vytvoriť boxplot, v ktorom nevidíme veľký rozdiel medzi fajčiarmi a nefajčiarmi.

In [ ]:
sns.boxplot(x='smoker', y='weight', data=data);

Keď si spravíme histogram, je očividné, že nefajčiarov je viac. To neprekáža, ale musíme si overiť či dáta pochádzajú z normálneho rozdelenia.

In [ ]:
sns.histplot(data=data, x='weight', hue='smoker');

Predtým ako použijeme test normálnosti sa vrátime k boxplotu, kde môžeme pozorovať veľké množstvo outlierov, ktoré je potrebné odstrániť. Použijeme teda metódu medzikvartilového rozdelenia (IQR), ktoré sa používa aj v boxplotoch. Táto metóda považuje za outliery všetky pozorovania, ktorých hodnoty sú väčšie ako 75-kvartil + 1.5 * IQR alebo menšie ako 25-kvartil - 1.5 * IQR.

In [ ]:
lower = data.weight.quantile(0.25) - 1.5 * stats.iqr(data.weight)
upper = data.weight.quantile(0.75) + 1.5 * stats.iqr(data.weight)

out = data[(data.weight > upper) | (data.weight < lower)]
temp = data.drop(out.index)
sns.histplot(data=temp, x='weight', hue='smoker');

Následne si môžeme spraviť test normálnosti. Shapiro-Wilk test nie je vhodný pre vzorky s veľkosťou väčšou ako 5000, na čo nás scipy aj upozornilo, takže urobíme druhý test použitím Kolmogorov-Smirnov testu. V tom prípade však musíme predpripraviť normálne rozdelenie pre danú premennú, nakoľko Kolmogorov-Smirnov test pri porovnávaní s normálnou distribúciou používa rozdelenie so stredom v 0 a štandardnou odchýlkou 1. Najprv teda nafittujeme hmotnosti fajčiarov a nefajčiarov do modelu normálneho rozdelenia a z neho získame stred a štandardnú odchýlku, ktorú použijeme v teste.

Ak si určíme p = 0.05, potom v Shapiro-Wilk teste fajčiari aj nefajčiari pochádzajú z iného ako normálneho rozdelenia, ale Kolmogorov-Sminov test nám vraví, že obe skupiny majú normálne rozdelenie. Keďže Shapiro-Wilk test nemusí byť presný pri veľkých datasetoch, budeme sa držať výsledku z Kolmogorov-Smirnov testu.

In [ ]:
smoker = temp.loc[temp.smoker == 1, 'weight']
non_smoker = temp.loc[temp.smoker == 0, 'weight']

print('Smoker:', stats.shapiro(smoker))
print('Non-smoker:', stats.shapiro(non_smoker))

In [ ]:
loc, scale = stats.norm.fit(smoker)
ns = stats.norm(loc=loc, scale=scale)

loc, scale = stats.norm.fit(non_smoker)
nns = stats.norm(loc=loc, scale=scale)

print('Smoker:', stats.kstest(smoker, ns.cdf))
print('Non-smoker:', stats.kstest(non_smoker, nns.cdf))

Keďže rozdelenia sú normálne, použijeme Studentov t-test. V tomto prípade je p > 0.01, takže výsledok je štatisticky významný a pôvodnú hypotézu nezamietame. Pre istotu môžeme použiť aj neparametrickú variantu t-testu (nakoľko Shapiro-Wilk test nám obe rozdelenia vyhodnotil ako rôzne od normálneho rozdelenia), konkrétne Mann-Whitney U-test, ktorý nám potvrdí ponechanie pôvodnej hypotézy.

In [ ]:
stats.ttest_ind(smoker, non_smoker)

In [ ]:
stats.mannwhitneyu(smoker, non_smoker)

Rozdiely si môžeme vizualizovať pomocou barplotu, kde vidíme, že rozdiel v hmotnosti medzi fajčiarmi a nefajčiarmi je zanedbateľný. Dôvod prečo na histograme vyzerali byť hodnoty rôzne je jednoduchý: nefajčiarov je v datasete viac ako fajčiarov. Štatisticky sme teda dokázali, že fajčenie má zanedbateľný vplyv na telesnú hmotnosť pacienta, minimálne v tejto vzorke dát.

In [ ]:
import statsmodels.stats.api as sms

print('Smoker:', sms.DescrStatsW(smoker).tconfint_mean())
print('Non-smoker:', sms.DescrStatsW(non_smoker).tconfint_mean())

sns.barplot(x='smoker', y='weight', data=temp[(temp.smoker == 0) | (temp.smoker == 1)], 
            capsize=0.1, errwidth=2, palette=sns.color_palette("Blues"));

## 2. Ovplyvňuje hodnota `hbver` to, či pacienti potrebujú pravidelné kontroly?

Z datasetu môžeme skúmať vplyv hodnoty premennej 'hbver' na pacientovu potrebnú ďaľšiu návštevu doktora. (danú premennou 'indicator') Definujme si teda nulovú a alternatívnu hypotézu:

H0: Pacienti, ktorí potrebujú pravidelné kontroly majú **rovnaké** hodnoty premennej `hbver` ako tí, ktorí pravidelné kontroly nepotrebujú.

H1: Pacienti, ktorí potrebujú pravidelné kontroly majú **vyššie/nižšie** hodnoty premennej `hbver` ako tí, ktorí pravidelné kontroly nepotrebujú.

Môžeme si vytvoriť boxplot, v ktorom vidíme, že ľudia ktorí nepotrebujú pravidelnú návštevu doktora majú viditeľne vyššie hodnoty premennej `hbver` ako tí, ktorí pravidelnú návštevu doktora potrebujú.

In [ ]:
sns.boxplot(x='indicator', y='hbver', data=data);

Keď si spravíme histogram, vidíme, že pacientov ktorí potrebujú pravidelnú návštevu doktora (`indicator=1`), je viac, ako pacientov ktorí nepotrebujú pravidelnú návštevu doktora (`indicator=0`). Vidíme, že histogramy pre obe skupiny sú navzájom posunuté, a ich výška je taktiež iná, čo je dané rozdielnym počtom pacientov s daným indikátorom (čo sme zistili vyššie pri boxplote). Poďme si teda urobiť test normálnosti.

In [ ]:
sns.histplot(data=data, x='hbver', hue='indicator')

V boxplote sme však mohli pozorovať vcelku veľké množstvo Outlierov, ktoré je z dát dobré odstrániť pre lepšie a konzistentnejšie výsledky. Toto odstránenie vykonáme pomocou medzikvartilového rozdelenia (IQR), ako v hypotéze vyššie. Táto metóda, ako bolo spomínané vyššie, považuje za outliery všetky pozorovania, ktorých hodnoty sú väčšie ako 75-kvartil + 1.5 * IQR alebo menšie ako 25-kvartil - 1.5 * IQR. Tu sme však mali od hypotézy 1 rozdiel, nakoľko bol boxplot pre každý z indikátorov s viditeľným rozdielom hodnôt `hbver`, museli sme vypočítať outlierov pre každú skupinu zvlášť, a až potom outlierov pre každý z indikátorov následne odstrániť.

In [ ]:
data_indicator_one = data.loc[data['indicator'] == 1]
data_indicator_zero = data.loc[data['indicator'] == 0]

lower_indicator_one = data_indicator_one.hbver.quantile(0.25) - 1.5 * stats.iqr(data_indicator_one.hbver)
upper_indicator_one = data_indicator_one.hbver.quantile(0.75) + 1.5 * stats.iqr(data_indicator_one.hbver)

lower_indicator_zero = data_indicator_zero.hbver.quantile(0.25) - 1.5 * stats.iqr(data_indicator_zero.hbver)
upper_indicator_zero = data_indicator_zero.hbver.quantile(0.75) + 1.5 * stats.iqr(data_indicator_zero.hbver)

out_one = data_indicator_one[(data_indicator_one.hbver > upper_indicator_one) | (data_indicator_one.hbver < lower_indicator_one)]
out_zero = data_indicator_zero[(data_indicator_zero.hbver > upper_indicator_zero) | (data_indicator_zero.hbver < lower_indicator_zero)]

vertical_cat = pd.concat([out_one, out_zero], axis=0)
temp = data.drop(vertical_cat.index)

# sns.boxplot(x='indicator', y='hbver', data=temp);
sns.histplot(data=temp, x='hbver', hue='indicator');

Pri Shapiro-Wilk teste sme boli upozornení, že údaje p-value nemusia byť presné pre počet záznamov viac ako 5000, čo je aj náš prípad. Zo Shapiro-Wilk testu nám vyšla iná ako normálna distribúcia, ale kvôli možnej nepresnosti si ešte urobíme Kolmogorov-Sminov test. A keďže Shapiro-Wilk test nemusí ukázať presne pri veľkých datasetoch, budeme sa riadiť výsledkom z Kolmogorov-Smirnov testu. Z Kolmogorov-Smirnov testu nám následne vyšiel výsledok že distribúcie je normálna, teda opak Shapiro-Wilk testu. Keďže Shapiro-Wilk nemusí ukazovať pri viac ako 5000 údajoch dobre, budeme považovať za správny Kolmogorov-Smirnov test, a teda že dáta pochádzajú z normálnej distribúcie

In [ ]:
temp_indicator_one = temp.loc[temp['indicator'] == 1, 'hbver']
temp_indicator_zero = temp.loc[temp['indicator'] == 0, 'hbver']

# smoker = temp.loc[temp.smoker == 1, 'weight']
# non_smoker = temp.loc[temp.smoker == 0, 'weight']

print('Indicator 1:', stats.shapiro(temp_indicator_one))
print('Indicator 0:', stats.shapiro(temp_indicator_zero))

In [ ]:
loc, scale = stats.norm.fit(temp_indicator_one)
ti_o = stats.norm(loc=loc, scale=scale)

loc, scale = stats.norm.fit(temp_indicator_zero)
ti_z = stats.norm(loc=loc, scale=scale)

print('Indicator 1:', stats.kstest(temp_indicator_one, ti_o.cdf))
print('Indicator 0:', stats.kstest(temp_indicator_zero, ti_z.cdf))

Keďže sme dostali normálne rozdelenie, použijeme Studentov t-test. V tomto prípade je p < 0.01, takže nulovú hypotézu zamietame a berieme do úvahy druhú, - alternatívnu hypotézu

In [ ]:
stat, p = stats.ttest_ind(temp_indicator_zero, temp_indicator_one)
print('Statistics=%.3f, p=%.5f' % (stat, p))

Rozdiely si môžeme vizualizovať pomocou barplotu, kde vidíme, že rozdiel v hodnotách `hbver` medzi ľuďmi s rozdielnym indikátorm je značne viditeľný. Štatisticky sme teda dokázali, že rozdielna hodnota premennej `indicator` má vplyv na výšku hodnôt premennej `hbver`, minimálne v nám poskytnutej vzorke dát. Ľudia, ktorí **nepotrebujú ďaľšiu návštevu doktora** (`indicator=0`) teda zvyčajne majú **vyššie hodnoty premennej** `hbver`

In [ ]:
import statsmodels.stats.api as sms

print('Indicator 1:', sms.DescrStatsW(temp_indicator_one).tconfint_mean())
print('Indicator 0:', sms.DescrStatsW(temp_indicator_zero).tconfint_mean())

sns.barplot(x='indicator', y='hbver', data=temp[(temp.indicator == 0) | (temp.indicator == 1)], 
            capsize=0.1, errwidth=2, palette=sns.color_palette("flare"));

# Experimentovanie s externými dátami

## Je niektorá krvná skupina náchylnejšia na CLL?

V prvej hypotéze sa vrátime ku krvným skupinám. Pomer krvných skupín v datasete bol približne rovnaký, ale pomer krvných skupín v populácií rovnaký nie je - napríklad skupina AB- sa vyskytuje iba pri 1% populácie, ale v grafe vyššie má približne rovnaký počet pacientov ako ostatné krvné skupiny.

Rozdelenie krvných skupín v populácii:

O Rh-positive --- 39 percent 

O Rh-negative ---  9 percent 

A Rh-positive --- 30 percent 

A Rh-negative ---  6 percent

B Rh-positive ---    9 percent

B Rh-negative ---   2 percent 

AB Rh-positive ---  4 percent 

AB Rh-negative --- 1 percent

Zdroj: https://www.aabb.org/for-donors-patients/faqs-about-blood-and-blood-donation#

Preto môže byť zaujímavé si vypočítať pomer krvných skupín v datasete s krvnými skupinami v populácii. Na nasledovnom grafe môžeme vidieť, že skupina AB- ďaleko prevyšuje ostatné krvné skupiny.

In [ ]:
'''
Function uses dictionary equivalent of switch statement that does not exist in Python.
Every blood group is divided by the proportion of blood group in population, which yields 
lower numbers in common blood groups and higher numbers in rare blood groups.
'''
def population_blood_group(ind, val):
    return {
        'A+': val / 0.30,
        'A-': val / 0.06,
        'B+': val / 0.09,
        'B-': val / 0.02,
        'AB+': val / 0.04,
        'AB-': val / 0.01,
        'O+': val / 0.39,
        'O-': val / 0.09
    }[ind]

# Copy the dataset, so we don't damage the original
temp = data.copy()

# Count the blood groups and reset index so we can use it in function
temp = temp.blood_group.value_counts().reset_index()

# Apply the function above to dataset
temp['blood_prop'] = temp.apply(lambda row: population_blood_group(row['index'], row.blood_group), axis=1)

# Normalize the values and plot the result
temp.set_index('index').apply(lambda x: x/x.max()).blood_prop.plot(xlabel='blood group', ylabel='proportion');

Krvné skupiny sa líšia v počte bielych a červených krviniek a krvných doštičiek, a tieto rozdiely môžu mať vplyv na rozdiely v korelácii s inými premennými. Pre zjednodušenie si negatívne a pozitívne varianty krvných skupín spojíme dokopy a skúsime vykresliť koreláciu s ostatnými premennými v datasete. 

In [ ]:
temp = data.copy()
temp.blood_group.replace({'A+': 'A', 'A-': 'A', 'B+': 'B', 'B-': 'B','O+': 'O', 'O-': 'O','AB+': 'AB', 'AB-': 'AB'}, inplace=True)
sns.heatmap(data=temp.groupby('blood_group').corr(), annot=True, fmt=".3f");

Graf je extrémne neprehľadný, takže vytvoríme samostatný podgraf pre každú krvnú skupinu. Môžeme vidieť pozitívnu koreláciu premenných `alt` a `erytrocyty`, ktorá je najsilnejšia pri krvnej skupine 0. Ďalej si môžeme všimnúť negatívnu koreláciu medzi premennými `hbver` a `indicator`, ktorá je približne rovnako silná pri všetkých krvných skupinách, čo nám úž môže napovedať, že premenná `hbver` môže mať vplyv na rozhodnutie, či pacient potrebuje pravidelné kontroly alebo nie.

Ďalšie slabšie pozitívne korelácie môžeme pozorovať medzi premennými `indicator` a `hematokrit`, `hemoglobin` a `alp` a negatívnu medzi `hbver` a `hematokrit`.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(30, 25))
sns.heatmap(data=temp[temp.blood_group == 'A'].corr(), ax=ax[0, 0], annot=True, fmt=".3f")
ax[0, 0].set_title('A')
sns.heatmap(data=temp[temp.blood_group == 'B'].corr(), ax=ax[0, 1], annot=True, fmt=".3f")
ax[0, 1].set_title('B')
sns.heatmap(data=temp[temp.blood_group == 'AB'].corr(), ax=ax[1, 0], annot=True, fmt=".3f")
ax[1, 0].set_title('AB')
sns.heatmap(data=temp[temp.blood_group == 'O'].corr(), ax=ax[1, 1], annot=True, fmt=".3f")
ax[1, 1].set_title('O');

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(15, 25))
temp.groupby('blood_group').leukocyty.plot(kind='kde', ax=ax[0, 0], legend=True)
temp.groupby('blood_group').erytrocyty.plot(kind='kde', ax=ax[1, 0], legend=True)
temp.groupby('blood_group').trombocyty.plot(kind='kde', ax=ax[2, 0], legend=True)
temp.groupby('blood_group').hemoglobin.plot(kind='kde', ax=ax[3, 0], legend=True)
temp.groupby('blood_group').alt.plot(kind='kde', ax=ax[0, 1], legend=True)
temp.groupby('blood_group').alp.plot(kind='kde', ax=ax[1, 1], legend=True)
temp.groupby('blood_group').ast.plot(kind='kde', ax=ax[2, 1], legend=True)
temp.groupby('blood_group').hbver.plot(kind='kde', ax=ax[3, 1], legend=True)

In [ ]:
temp.groupby('blood_group').indicator.value_counts().plot(kind='bar')